In [1]:
import pandas as pd
import numpy as np
import seaborn as sb

data = pd.read_pickle('binning1.pickle')

def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['Loan_Status'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w

dt_train = pd.DataFrame()
dt_train['Loan_Status'] = data['Loan_Status']
ATTRS = ['Education','Self_Employed','Credit_History','Property_Area','APPLICANTINCOME_BIN','COAPPLICANTINCOME_BIN','LOANAMOUNT_BIN','LOAN_AMOUNT_TERM_BIN']


NEW_WOE_ATTRS=[]
for attr in ATTRS:
    attr_element = set(data[attr].to_numpy())
    attr_woe = dict()
    for i in attr_element:
        w = WOE(data, attr, i)
        attr_woe[i] = w
        NEW_WOE_ATTRS.append( attr_woe[i])
    print(f'WOE {attr} -> {attr_woe}')
    

    lst = []
    for s in data[attr]:
        woe = attr_woe[s]
        lst.append(woe)

    new_field = attr + '_WOE'
    dt_train[new_field] = lst


dt_train

WOE Education -> {0: -0.3024010214130441, 1: 0.08707299360422699}
WOE Self_Employed -> {0: -0.10702026707619076, 1: 0.016773950270473608}
WOE Credit_History -> {0.0: -3.175073202209808, 1.0: 0.5415430068757282}
WOE Property_Area -> {1: -0.1398531571486983, 2: -0.3938413838140067, 3: 0.48947415124459764}
WOE APPLICANTINCOME_BIN -> {Interval(9327.5, 81000.0, closed='right'): -0.28008249845734856, Interval(2222.6, 2601.8, closed='right'): 0.16670075898154685, Interval(4352.4, 5179.6, closed='right'): -0.2229240846174, Interval(2601.8, 3055.0, closed='right'): 0.2749143436217797, Interval(3418.0, 3848.0, closed='right'): -0.16229946280096516, Interval(6250.0, 9327.5, closed='right'): 0.03375228680489156, Interval(3848.0, 4352.4, closed='right'): 0.06360524995457266, Interval(149.999, 2222.6, closed='right'): -0.005387621532378887, Interval(3055.0, 3418.0, closed='right'): 0.09259278682782489, Interval(5179.6, 6250.0, closed='right'): 0.09259278682782489}
WOE COAPPLICANTINCOME_BIN -> {Inter

,Loan_Status,Education_WOE,Self_Employed_WOE,Credit_History_WOE,Property_Area_WOE,APPLICANTINCOME_BIN_WOE,COAPPLICANTINCOME_BIN_WOE,LOANAMOUNT_BIN_WOE,LOAN_AMOUNT_TERM_BIN_WOE
1,0,0.087073,0.016774,0.541543,-0.393841,-0.222924,-0.035241,0.200806,0.033999
2,1,0.087073,-0.107020,0.541543,-0.139853,0.274914,-0.029198,-0.005388,0.033999
3,1,-0.302401,0.016774,0.541543,-0.139853,0.166701,0.457236,0.562596,0.033999
4,1,0.087073,0.016774,0.541543,-0.139853,0.092593,-0.029198,0.274914,0.033999
5,1,0.087073,-0.107020,0.541543,-0.139853,0.092593,-0.191175,-0.523593,0.033999
...,...,...,...,...,...,...,...,...,...
495,1,0.087073,0.016774,0.541543,0.489474,0.092593,-0.029198,0.166701,0.117285
496,1,-0.302401,0.016774,0.541543,-0.393841,0.166701,0.120764,0.166701,0.033999
497,1,0.087073,0.016774,0.541543,-0.139853,-0.222924,-0.130551,-0.069926,0.117285
498,1,0.087073,-0.107020,0.541543,0.489474,0.274914,-0.029198,-0.005388,0.033999


In [2]:
Y =  dt_train['Loan_Status'].to_numpy()
x_1 = dt_train['Education_WOE'].to_numpy()
x_2 = dt_train['Self_Employed_WOE'].to_numpy()
x_3 = dt_train['Credit_History_WOE'].to_numpy()
x_4 = dt_train['Property_Area_WOE'].to_numpy()
x_5 = dt_train['APPLICANTINCOME_BIN_WOE'].to_numpy()
x_6 = dt_train['COAPPLICANTINCOME_BIN_WOE'].to_numpy()
x_7 = dt_train['LOANAMOUNT_BIN_WOE'].to_numpy()
x_8 = dt_train['LOAN_AMOUNT_TERM_BIN_WOE'].to_numpy()


array_ones = np.ones(x_1.shape)

X = np.array([array_ones, x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8])
X = np.transpose(X)
X

array([[ 1.        ,  0.08707299,  0.01677395, ..., -0.03524058,
         0.20080637,  0.03399862],
       [ 1.        ,  0.08707299, -0.10702027, ..., -0.02919827,
        -0.00538762,  0.03399862],
       [ 1.        , -0.30240102,  0.01677395, ...,  0.4572359 ,
         0.56259642,  0.03399862],
       ...,
       [ 1.        ,  0.08707299,  0.01677395, ..., -0.13055076,
        -0.06992614,  0.1172854 ],
       [ 1.        ,  0.08707299, -0.10702027, ..., -0.02919827,
        -0.00538762,  0.03399862],
       [ 1.        ,  0.08707299,  0.01677395, ..., -0.19117539,
        -0.09977911,  0.03399862]])

In [3]:
np.matmul(np.transpose(X), X)

array([[ 4.82000000e+02,  2.24283339e+00,  1.62214120e-01,
         4.57721089e+00,  1.35906159e+01,  2.59456082e+00,
         2.70867280e+00,  7.51721474e+00,  2.52455738e+00],
       [ 2.24283339e+00,  1.22085789e+01,  2.27614897e-02,
         9.28305886e+00,  1.96679080e+00, -1.55757390e+00,
        -7.01978889e-01, -3.22740316e+00,  6.48969565e-02],
       [ 1.62214120e-01,  2.27614897e-02,  8.50624345e-01,
        -7.29148717e-02,  1.72983690e-01,  3.12126504e-01,
         1.37294418e-01,  3.22419839e-01, -1.48447651e-01],
       [ 4.57721089e+00,  9.28305886e+00, -7.29148717e-02,
         8.16715225e+02,  1.25349493e+01, -1.41910741e+00,
         2.77935693e+00, -2.95568725e+00,  1.19893522e+01],
       [ 1.35906159e+01,  1.96679080e+00,  1.72983690e-01,
         1.25349493e+01,  6.84828255e+01,  2.71348307e-01,
         1.42781314e+00,  2.28226636e-02, -1.50044201e+00],
       [ 2.59456082e+00, -1.55757390e+00,  3.12126504e-01,
        -1.41910741e+00,  2.71348307e-01,  1.351823

In [4]:
np.linalg.inv(np.matmul(np.transpose(X), X))

array([[ 2.09832292e-03, -4.75377385e-04, -4.78052175e-05,
         1.84240573e-06, -4.01761786e-04, -3.08725004e-04,
        -2.23608232e-04, -3.86697569e-04, -2.18797231e-04],
       [-4.75377385e-04,  8.58086955e-02, -7.31777142e-03,
        -9.14577383e-04, -2.24811485e-03,  8.03970810e-03,
         1.83840779e-03,  6.09447084e-03,  3.76784709e-04],
       [-4.78052175e-05, -7.31777142e-03,  1.19014542e+00,
         1.01194476e-04, -2.44639581e-03, -2.50971610e-02,
        -6.59583938e-03, -6.86458742e-03,  5.45274225e-03],
       [ 1.84240573e-06, -9.14577383e-04,  1.01194476e-04,
         1.24682306e-03, -2.08080082e-04,  4.13165392e-05,
        -2.90543488e-04,  3.46058820e-05, -5.19931351e-04],
       [-4.01761786e-04, -2.24811485e-03, -2.44639581e-03,
        -2.08080082e-04,  1.48451714e-02, -2.21122050e-04,
        -1.61237156e-03,  1.23333947e-04,  9.10870599e-04],
       [-3.08725004e-04,  8.03970810e-03, -2.50971610e-02,
         4.13165392e-05, -2.21122050e-04,  7.860602

In [5]:
np.matmul(np.transpose(X), Y)

array([335.0, 4.243115896310715, 0.29612199470208794, 159.1172100488564,
       23.320629258993247, 4.681675421454834, 4.5028953342744975,
       13.431871604245462, 8.600921416634845], dtype=object)

In [6]:
B = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.transpose(X), Y))
B

array([0.6823025959804921, 0.13574371533334223, 0.0719108441204784,
       0.18518303895210253, 0.16943134452500655, 0.18118108870169602,
       0.04164801787390304, 0.21645128240619993, 0.17972580801252877],
      dtype=object)

In [7]:
def predict(x):
    y = B[0] + B[1]*x+ B[2]*x+ B[3]*x+ B[4]*x+ B[5]*x+ B[6]*x+ B[7]*x+ B[8]*x
    return y


eq = f'y = {B[0]:.3f}'
for i in range(1, len(B)):
    eq = eq + f'+ ({B[i]:.3f})x_{i}'


print(eq)



y = 0.682+ (0.136)x_1+ (0.072)x_2+ (0.185)x_3+ (0.169)x_4+ (0.181)x_5+ (0.042)x_6+ (0.216)x_7+ (0.180)x_8


In [8]:
dt_train['predict'] = dt_train['predict'].apply(predict)

KeyError: 'predict'

In [ ]:
# y_predict
dt_train['y_predict'] = B[0] + B[1]*dt_train['Education_WOE'] + B[2]*dt_train['Self_Employed_WOE'] \
        + B[3]*dt_train['Credit_History_WOE'] + B[4]*dt_train['Property_Area_WOE'] + B[5]*dt_train['APPLICANTINCOME_BIN_WOE'] \
        + B[6]*dt_train['COAPPLICANTINCOME_BIN_WOE'] + B[7]*dt_train['LOANAMOUNT_BIN_WOE'] + B[8]*dt_train['LOAN_AMOUNT_TERM_BIN_WOE']

dt_train['P'] = 1/(1+np.exp(-dt_train['y_predict']))

dt_train['predict'] = dt_train['P'].map(lambda x: 1 if x >= 0.5 else 0)



# dt_train['P(y_predict)'] = 

# s_predict

dt_train

In [ ]:
n_correct = (dt_train['Loan_Status'] == dt_train['predict']).sum()
n_correct

In [ ]:
Predict_y('1')

In [ ]:
dt_train['P(y_predict)'] = dt_train['y_predict'].apply(Predict_y)
dt_train

In [ ]:
n_correct = (dt_train['Loan_Status'] == dt_train['s_predict']).sum()
print('correct is')
print(n_correct)


accuracy = n_correct/482
print('accuracy is')
print(accuracy)